In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.metrics import accuracy_score
warnings.filterwarnings("ignore")
from sklearn.tree import  DecisionTreeClassifier
from sklearn import tree
from sklearn.model_selection import train_test_split

In [ ]:
df=pd.read_csv("/content/Company_Data.csv")
df.shape
df.info()

In [ ]:
df.head()


In [ ]:
#For three categories lets divide the Sales values with Quartiles , less than first quartile as Low , between Second Quartile as Medium and above Third Quartile as High
df.Sales.max()

16.270/3

In [ ]:
16.270/3

In [ ]:
5.423333333333333*2

In [ ]:
5.423333333333333*3

In [ ]:
df1=df.copy()
df1['Sales_cat'] = pd.cut(x = df1['Sales'], bins = [0,5.39,9.46,17], labels=['Low','Medium','High'], right = False)
df1.head()

In [ ]:
df1.Sales_cat.value_counts()

In [ ]:
sns.heatmap(df.corr(), annot=True, fmt='.2g', cmap="viridis",  linewidths=0.5, linecolor='black')

In [ ]:
ot=df1.copy()
fig, axes=plt.subplots(8,1,figsize=(14,10),sharex=False,sharey=False)
sns.boxplot(x='Sales',data=ot,palette='crest',ax=axes[0])
sns.boxplot(x='CompPrice',data=ot,palette='crest',ax=axes[1])
sns.boxplot(x='Income',data=ot,palette='crest',ax=axes[2])
sns.boxplot(x='Advertising',data=ot,palette='crest',ax=axes[3])
sns.boxplot(x='Population',data=ot,palette='crest',ax=axes[4])
sns.boxplot(x='Price',data=ot,palette='crest',ax=axes[5])
sns.boxplot(x='Age',data=ot,palette='crest',ax=axes[6])
sns.boxplot(x='Education',data=ot,palette='crest',ax=axes[7])
plt.tight_layout(pad=2.0)

In [ ]:
#Outliers
"""removing the ouliers"""
# List of column names with continuous variables
continuous_columns = [  "CompPrice" , "Price" ]
# Create a new DataFrame without outliers for each continuous column
data_without_outliers = df1.copy()
for columns in continuous_columns:
    Q1 = data_without_outliers[columns].quantile(0.25)
    Q3 = data_without_outliers[columns].quantile(0.75)
    IQR = Q3 - Q1
    lower_whisker_Length = Q1 - 1.5 * IQR
    upper_whisker_Length = Q3 + 1.5 * IQR
    data_without_outliers = data_without_outliers[(data_without_outliers[columns] >= lower_whisker_Length) & (data_without_outliers[columns]<= upper_whisker_Length)]
# Print the cleaned data without outliers
print(data_without_outliers)
df1 = data_without_outliers

In [ ]:
df1.drop("Sales",axis=1 ,inplace=True)

In [ ]:
#Label Encoding
from sklearn.preprocessing import LabelEncoder
LE=LabelEncoder()

df1["ShelveLoc"]=LE.fit_transform(df1["ShelveLoc"])
df1["Urban"]=LE.fit_transform(df1["Urban"])
df1["US"]=LE.fit_transform(df1["US"])
df1["Sales_cat"]=LE.fit_transform(df1["Sales_cat"])

df1.head()

In [ ]:
#Data Spliting

X=df1.iloc[:,0:10]
Y=df1[["Sales_cat"]]


In [ ]:
# data partition
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test  = train_test_split(X,Y, test_size=0.30)


from sklearn.tree import DecisionTreeClassifier
DT = DecisionTreeClassifier(max_depth=6)

DT.fit(X_train,Y_train) # Bo + b1x1 + B2x2
Y_pred_train = DT.predict(X_train)
Y_pred_test = DT.predict(X_test)
training_accuracy = accuracy_score(Y_train,Y_pred_train).round(3)
test_accuracy = accuracy_score(Y_test,Y_pred_test).round(3)

print(training_accuracy)
print(test_accuracy)

print("Number of Nodes",DT.tree_.node_count)
print("Level of Depth",DT.tree_.max_depth)


In [ ]:
# validation set method --> validaton hold-off method
training_accuracy = []
test_accuracy = []
Tr = []
Ts = []

DT = DecisionTreeClassifier(max_depth=6)


for i in range(1,101,1):
    X_train,X_test,Y_train,Y_test  = train_test_split(X,Y, test_size=0.30, random_state=i)
    DT.fit(X_train,Y_train)
    Y_pred_train = DT.predict(X_train)
    Y_pred_test = DT.predict(X_test)
    training_accuracy.append(accuracy_score(Y_train,Y_pred_train).round(3))
    test_accuracy.append(accuracy_score(Y_test,Y_pred_test).round(3))

import numpy as np
print("Average trianing accuray",np.mean(training_accuracy).round(2))
print("Average test accuray",np.mean(test_accuracy).round(2))


In [ ]:
#Building Decision Tree Classifier using Entropy Criteria with Alpha values Post Pruning
df1_entropy = DecisionTreeClassifier(criterion='entropy', random_state=0)
df1_entropy.fit(X_train,Y_train)

Y_pred_train = df1_entropy.predict(X_train)
Y_pred_test = df1_entropy.predict(X_test)
training_accuracy = accuracy_score(Y_train,Y_pred_train).round(3)
test_accuracy = accuracy_score(Y_test,Y_pred_test).round(3)

print(training_accuracy)
print(test_accuracy)


In [ ]:
plt.figure(figsize=(12,8),dpi=500)
tree.plot_tree(df1_entropy,filled=True)
plt.show()

In [ ]:
path = df1_entropy.cost_complexity_pruning_path(X_train, Y_train)
ccp_alphas = path.ccp_alphas
impurities = path.impurities

clfs = []
for ccp_alpha in ccp_alphas:
    clf = DecisionTreeClassifier(criterion='entropy',random_state=0, ccp_alpha=ccp_alpha)
    clf.fit(X_train, Y_train)
    clfs.append(clf)
print("Number of nodes in the last tree is: {} with ccp_alpha: {}".format(clfs[-1].tree_.node_count, ccp_alphas[-1]))

In [ ]:
#Accuracy vs alpha for training and testing sets

train_scores = [clf.score(X_train, Y_train) for clf in clfs]
test_scores = [clf.score(X_test, Y_test) for clf in clfs]

fig, ax = plt.subplots(figsize=(16,9))
ax.set_xlabel("alpha")
ax.set_ylabel("accuracy")
ax.set_title("Accuracy vs alpha for training and testing sets")
ax.plot(ccp_alphas, train_scores, marker='o', label="train",
        drawstyle="steps-post")
ax.plot(ccp_alphas, test_scores, marker='o', label="test",
        drawstyle="steps-post")
ax.legend()
plt.show()

In [ ]:
#Choosing the best value of cc_alpha where the testing and training accuracy are almost same or near or closer to each other
clf = DecisionTreeClassifier(criterion='entropy',random_state=0, ccp_alpha=0.02876)
clf.fit(X_train,Y_train)

Y_pred_train = clf.predict(X_train)
Y_pred_test = clf.predict(X_test)
training_accuracy = accuracy_score(Y_train,Y_pred_train).round(3)
test_accuracy = accuracy_score(Y_test,Y_pred_test).round(3)

print(training_accuracy)
print(test_accuracy)


In [ ]:
plt.figure(figsize=(15,10))
tree.plot_tree(clf,filled=True)
plt.show()

In [ ]:
#Building Decision Tree Classifier using Gini Criteria with Alpha values Post Pruning

df1_gini = DecisionTreeClassifier(criterion='gini', random_state=0)
df1_gini.fit(X_train,Y_train)

Y_pred_train = df1_gini.predict(X_train)
Y_pred_test = df1_gini.predict(X_test)
training_accuracy = accuracy_score(Y_train,Y_pred_train).round(3)
test_accuracy = accuracy_score(Y_test,Y_pred_test).round(3)

print(training_accuracy)
print(test_accuracy)



In [ ]:
plt.figure(figsize=(12,8),dpi=500)
tree.plot_tree(df1_gini,filled=True)
plt.show()

In [ ]:
path = df1_gini.cost_complexity_pruning_path(X_train, Y_train)
ccp_alphas = path.ccp_alphas
impurities = path.impurities

clfs = []
for ccp_alpha in ccp_alphas:
    clf = DecisionTreeClassifier(criterion='gini',random_state=0, ccp_alpha=ccp_alpha)
    clf.fit(X_train, Y_train)
    clfs.append(clf)
print("Number of nodes in the last tree is: {} with ccp_alpha: {}".format(clfs[-1].tree_.node_count, ccp_alphas[-1]))

In [ ]:
#Accuracy vs alpha for training and testing sets

train_scores = [clf.score(X_train, Y_train) for clf in clfs]
test_scores = [clf.score(X_test, Y_test) for clf in clfs]

fig, ax = plt.subplots(figsize=(16,9))
ax.set_xlabel("alpha")
ax.set_ylabel("accuracy")
ax.set_title("Accuracy vs alpha for training and testing sets")
ax.plot(ccp_alphas, train_scores, marker='o', label="train",
        drawstyle="steps-post")
ax.plot(ccp_alphas, test_scores, marker='o', label="test",
        drawstyle="steps-post")
ax.legend()
plt.show()

In [ ]:
#Choosing the best value of cc_alpha where the testing and training accuracy are almost same or near or closer to each other
clf = DecisionTreeClassifier(criterion='gini',random_state=0, ccp_alpha=0.01567)
clf.fit(X_train,Y_train)

Y_pred_train = clf.predict(X_train)
Y_pred_test = clf.predict(X_test)
training_accuracy = accuracy_score(Y_train,Y_pred_train).round(3)
test_accuracy = accuracy_score(Y_test,Y_pred_test).round(3)

print(training_accuracy)
print(test_accuracy)

In [ ]:
plt.figure(figsize=(15,10))
tree.plot_tree(clf,filled=True)
plt.show()